In [1]:
#import packages
from datetime import datetime
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
from pandasql import sqldf
import re
import gc
import zipfile
from zipfile import ZipFile
from io import StringIO
import sys

from os import listdir
from os.path import isfile, join
import psutil

from collections import Counter
import linecache
import tracemalloc

import reshape


In [2]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [3]:
#get ticker, company name and CIK numbers 
ticker_url = "https://www.sec.gov/include/ticker.txt"
ticker_request = requests.get(ticker_url).content
ticker_df = pd.read_csv(io.StringIO(ticker_request.decode('utf-8')),sep="\t")
ticker_df.columns = ['ticker', 'cik']
ticker_df["ticker"] = ticker_df["ticker"].str.lower()


#get sp500 tickers 
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = pd.DataFrame(sp500[0]['Symbol'])
sp500.columns = ['ticker']
sp500["ticker"] = sp500["ticker"].str.lower()
#print(sp500)

#get cik of sp 500
sample = pd.merge(ticker_df,sp500,on='ticker')
print(sample )

    ticker      cik
0      aal     6201
1      aap  1158449
2     aapl   320193
3     abbv  1551152
4      abc  1140859
..     ...      ...
495    yum  1041061
496    zbh  1136869
497   zbra   877212
498   zion   109380
499    zts  1555280

[500 rows x 2 columns]


In [4]:
#set up script to traverse directory of quarterly submissions
def generate_finance_table_csv():

    #store all finances in a table
    finance_list = []

    #go into directory with data
    file_os = ''
    
    start = time.time()
    for i in os.listdir():
        file_path = i
        #if its a quarterly filing data, parse it
        if 'q1' in file_path or 'q2' in file_path or 'q3' in file_path or 'q4' in file_path:
            try:
                tracemalloc.start()
                print("starting: " + file_path)

                sub = pd.read_csv(file_path + '/' + 'sub.txt', sep = '\t', encoding = "ISO-8859-1", iterator = True, chunksize =100000)
                sub = pd.concat(sub)
                #clean up form column and filter to just 10-K submissions
                sub['form'] = sub['form'].replace(np.nan, '', regex=True)
                sub = sub[sub['form'] == '10-K']
                #filter to the s&p500
                sub = pd.merge(sub,sample, on = ['cik'])

                pre = pd.read_csv(file_path + '/' + 'pre.txt', sep = '\t', encoding = "ISO-8859-1", chunksize =1000000) 
                num = pd.read_csv(file_path + '/' + 'num.txt', sep = '\t', encoding = "ISO-8859-1",  chunksize =100000)

                print("joining num to sub")
                num_sub = pd.DataFrame()              
                #join in numbers to submissions in chunks
                for chunks in num: 
                    #print(chunks)
                    num_sub = pd.concat([num_sub, sub.merge(chunks, on=['adsh'])])

                #no longer need sub
                del sub

                print("joining pre to numsub")
                nsp = pd.DataFrame()
                #add in plabel and stmt info
                for chunks in pre:
                    #print(chunks)
                    nsp = pd.concat([nsp, num_sub.merge(chunks, on=['adsh', 'tag', 'version'])])
                    nsp = nsp[nsp['stmt'].isin(['BS', 'IS', 'CF', 'CI', 'EQ'])]
                # delete files no longer needed
                del num_sub
                del pre
                del num


                #print(nsp.columns)

                nsp = nsp[['name', 'sic', 'fye', 'form', 'period', 
                           'fy', 'fp', 'filed','ticker', 'cik','ddate', 
                           'qtrs', 'uom', 'value', 'adsh','stmt',  'tag', 
                           'version', 'plabel']]

                #add finances to master table
                finance_list.append(nsp)
                del nsp
                print("processed_data: " + file_path)
                gc.collect()
                snapshot = tracemalloc.take_snapshot()
                display_top(snapshot)
            except Exception as ex:
                print("failed: " + file_path)
                print(str(ex))
    
    #switch back to github directory 
    
    finance_table = pd.concat(finance_list)
    del finance_list 
    finance_table.to_csv("total_finance_table.csv")
    end = time.time()
    print("finished traversing files in X secs")
    print(end - start)
    #return(finance_table)

In [5]:
# #run function to generate finance table csv

generate_finance_table_csv()

starting: 2014q1
joining num to sub
joining pre to numsub
processed_data: 2014q1
Top 3 lines
#1: core\algorithms.py:1657: 25078.3 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 14015.5 KiB
    data = self._reader.read(nrows)
#3: internals\managers.py:1848: 7388.1 KiB
    stacked = np.empty(shape, dtype=dtype)
271 other: 1876.7 KiB
Total allocated size: 48358.6 KiB
starting: 2014q2
joining num to sub
joining pre to numsub
processed_data: 2014q2
Top 3 lines
#1: core\algorithms.py:1657: 26137.8 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 16286.1 KiB
    data = self._reader.read(nrows)
#3: internals\managers.py:1848: 11092.2 KiB
    stacked = np.empty(shape, dtype=dtype)
336 other: 2436.7 KiB
Total allocated size: 55952.8 KiB
starting: 2014q3
joining num to sub
joining pre to numsub
processed_data: 2014q3
Top 3 lines
#1: core\algorithms.py:1657: 27837.1 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 11634.5 KiB
    

c:\users\daniel\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


joining pre to numsub
processed_data: 2015q1
Top 3 lines
#1: core\algorithms.py:1657: 55402.8 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 15639.8 KiB
    data = self._reader.read(nrows)
#3: internals\managers.py:1848: 6313.8 KiB
    stacked = np.empty(shape, dtype=dtype)
394 other: 4841.6 KiB
Total allocated size: 82198.0 KiB
starting: 2015q2
joining num to sub
joining pre to numsub
processed_data: 2015q2
Top 3 lines
#1: core\algorithms.py:1657: 55495.7 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 10074.6 KiB
    data = self._reader.read(nrows)
#3: indexes\base.py:763: 3963.8 KiB
    taken = self.values.take(indices)
394 other: 3478.5 KiB
Total allocated size: 73012.6 KiB
starting: 2015q3
joining num to sub
joining pre to numsub
processed_data: 2015q3
Top 3 lines
#1: core\algorithms.py:1657: 57183.8 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 9092.3 KiB
    data = self._reader.read(nrows)
#3: indexes\base.p

<ipython-input-5-228edc6bb70e>:3: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_finance_table_csv()


joining num to sub
joining pre to numsub
processed_data: 2020q2
Top 3 lines
#1: core\algorithms.py:1657: 191545.7 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 30057.1 KiB
    data = self._reader.read(nrows)
#3: indexes\base.py:763: 13679.2 KiB
    taken = self.values.take(indices)
471 other: 6831.2 KiB
Total allocated size: 242113.2 KiB
finished traversing files in X secs
634.8846199512482


In [6]:
#main function goes through and tries to parse out the historical data we need: VERY LONG 

def parse_data(df):
    gc.collect()
    ############################################################
    #process data a bit 
    
    #lower case plabel 
    df['plabel'] = df['plabel'].str.lower()
    
    #turn things into an integer
    df['qtrs'] = df.qtrs.astype(int)
    df['period'] = df.period.astype(int)
    df['fy'] = df.fy.astype(int)
    df['fye'] = df.fye.astype(int)
    df['ddate'] = df.ddate.astype(int)
    df['ddate_prev'] = df.ddate.astype(int) + 10000
    
    #turn things to strings
    df['period'] = df['period'].astype(str)
    df['fye'] = df['fye'].astype(str)
    df['fy'] = df['fy'].astype(str)
    df['ddate'] = df['ddate'].astype(str)
    df['ddate_prev'] = df['ddate_prev'].astype(str)
    #pad fye with a leading 0 
    df['fye'] = df['fye'].apply(lambda x: x.zfill(4))
    
    #fill nas inplace = true changes the df directly
    df['period'].fillna('', inplace=True)
    df['fy'].fillna('', inplace=True)
    df['fye'].fillna('', inplace=True)
    df['plabel'].fillna('', inplace=True)
    df['ddate'].fillna('', inplace=True)
    df['ddate_prev'].fillna('', inplace=True)
    
    #create a joinable key
    df['id'] = df['fy'].astype(str) + '_' + df['cik'].astype(str)

    #get distinct company and years
    key_df = pd.DataFrame(df.id.unique(), columns = ['id'])
    gc.collect()
    
    #######################################################################################################
    #get annual revenue 
    revenue_rows = df[
        # get yearly cash flow statements
        (df.stmt.str.contains('IS',na=False))
        & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('revenue',na=False) | df.plabel.str.contains('sales',na=False)) 
    ]
    #get max value for each company and fiscal year
    revenue_agg = revenue_rows.groupby(['id'], as_index=False)['value'].max()  
    revenue_agg.columns = ['id', 'revenue']
    #print(revenue_agg)
    del revenue_rows
    
    #get annual cash flow from operations rows 
    cfo_rows = df[
        # get yearly cash flow statements
        (df.stmt.str.contains('CF',na=False))
        & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('cash',na=False)) 
        & (df.plabel.str.contains('operating',na=False)) 
        ### exclusion words #####
        & (~df.plabel.str.contains('discontinue', na=False))
        & (~df.plabel.str.contains('cost', na=False))
        & (~df.plabel.str.contains('lease', na=False))
        & (~df.plabel.str.contains('adjustments', na=False))
        & (~df.plabel.str.contains('non-cash', na=False))
        & (~df.plabel.str.contains('other', na=False))
        & (~df.plabel.str.contains('disposal', na=False))
        & (~df.plabel.str.contains('escrow', na=False))
        & (~df.plabel.str.contains('investing', na=False))
        & (~df.plabel.str.contains('partnership', na=False))
        & (~df.plabel.str.contains('restricted', na=False))
        & (~df.plabel.str.contains('securities', na=False))
    ]
    #get max value for each company and fiscal year
    cfo_agg = cfo_rows.groupby(['id'], as_index=False)['value'].max()  
    cfo_agg.columns = ['id', 'cfo']
    #print(cfo_agg)
    del cfo_rows
    
    #get annual shares outstanding 
    shares_outstanding_rows = df[
        #get yearly income stmt 
        (df.stmt.str.contains('IS',na=False))
       # & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.uom.str.contains('shares',na=False))
        #inclusion words
        & (df.plabel.str.contains('shares',na=False) | df.plabel.str.contains('stock',na=False) )
        ### exclusion words
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    shares_outstanding_agg = shares_outstanding_rows.groupby(['id'], as_index=False)['value'].max()  
    shares_outstanding_agg.columns = ['id', 'shares_outstanding']
    del shares_outstanding_rows
    
    #get current assets 
    current_assets_rows = df[
        #get yearly balance sheet statements
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('current',na=False)) 
        & (df.plabel.str.contains('asset',na=False)) 
    ]
    current_assets_agg = current_assets_rows.groupby(['id'], as_index=False)['value'].max()  
    current_assets_agg.columns = ['id', 'current_assets']
    del current_assets_rows
    
    #get current liabilites
    current_liabilities_rows = df[
        #get yearly balance sheet statements
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('current',na=False)) 
        & (df.plabel.str.contains('liabilities',na=False)) 
    ]
    current_liabilities_agg = current_liabilities_rows.groupby(['id'], as_index=False)['value'].max()  
    current_liabilities_agg.columns = ['id', 'current_liabilities']
    del current_liabilities_rows
    
    #get depreciation and amortization
    dep_amort_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('depreciation',na=False)) 
        & (df.plabel.str.contains('amortization',na=False)) 
    ]
    dep_amort_agg = dep_amort_rows.groupby(['id'], as_index=False)['value'].max()  
    dep_amort_agg.columns = ['id', 'dep_amort']
    del dep_amort_rows
    
    # try to get depreciation only 
    dep_only_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('depreciation',na=False)) 
    ]
    dep_only_agg = dep_only_rows.groupby(['id'], as_index=False)['value'].max()  
    dep_only_agg.columns = ['id', 'dep_only']
    del dep_only_rows
    
    # try to get amort only
    amort_only_rows = df[
        #get yearly balance sheet statements
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('amortization',na=False)) 
    ]
    amort_only_agg = amort_only_rows.groupby(['id'], as_index=False)['value'].max()  
    amort_only_agg.columns = ['id', 'amort_only']
    del amort_only_rows
    
    #get PPE 
    ppe_rows = df[
        #get yearly income statements
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('property',na=False)) 
        & (df.plabel.str.contains('equipment',na=False)) 
    ]
    ppe_agg = ppe_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_agg.columns = ['id', 'ppe']
    del ppe_rows
    
    #get PPE of the year before
    ppe_prev_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate_prev'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('property',na=False)) 
        & (df.plabel.str.contains('equipment',na=False)) 
    ]
    ppe_prev_agg = ppe_prev_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_prev_agg.columns = ['id', 'ppe_prev']
    del ppe_prev_rows
    
    #get operating income 
    operating_income_rows = df[
        #get yearly income stmt 
        (df.stmt.str.contains('IS',na=False))
        & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('operating',na=False))
        & (df.plabel.str.contains('income',na=False) )
        ### exclusion words
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    operating_income_agg = operating_income_rows.groupby(['id'], as_index=False)['value'].max()  
    operating_income_agg.columns = ['id', 'operating_income']
    del operating_income_rows
    
    #get long term debt on balance sheet
    lt_debt_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('debt',na=False)) 
    ]
    lt_debt_agg = lt_debt_rows.groupby(['id'], as_index=False)['value'].max()  
    lt_debt_agg.columns = ['id', 'lt_debt']
    del lt_debt_rows
    
    #get short term debt on balance sheet
    st_debt_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('st_debt',na=False)) 
    ]
    st_debt_agg = st_debt_rows.groupby(['id'], as_index=False)['value'].max()  
    st_debt_agg.columns = ['id', 'st_debt']
    del st_debt_rows
    
    #get short term debt proceeds from cash flow stmt
    st_debt_proceeds_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        & (df.plabel.str.contains('short',na=False))
        & (df.plabel.str.contains('debt',na=False)) 
        & (df.plabel.str.contains('proceed',na=False))
    ]
    st_debt_proceeds_agg = st_debt_proceeds_rows.groupby(['id'], as_index=False)['value'].max()  
    st_debt_proceeds_agg.columns = ['id', 'st_debt_proceeds']
    del st_debt_proceeds_rows
    
    #get long term debt proceeds  from cash flow stmt
    lt_debt_proceeds_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        &
        (
            (
                (df.plabel.str.contains('long',na=False))
                & (df.plabel.str.contains('debt',na=False)) 
                & (df.plabel.str.contains('proceed',na=False))
            ) | 
            (
            
                (df.plabel.str.contains('issuance',na=False)) 
                & (df.plabel.str.contains('senior notes',na=False))
                & (df.plabel.str.contains('proceed',na=False))
            )
        )
    ]
    
    
    lt_debt_proceeds_agg = lt_debt_proceeds_rows.groupby(['id'], as_index=False)['value'].max()  
    lt_debt_proceeds_agg.columns = ['id', 'lt_debt_proceeds']
    del lt_debt_proceeds_rows
    
    #get short term debt repayments from cash flow stmt
    st_debt_repayments_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        & (df.plabel.str.contains('short',na=False))
        & (df.plabel.str.contains('debt',na=False)) 
        & (df.plabel.str.contains('repayment',na=False))
    ]
    st_debt_repayments_agg = st_debt_repayments_rows.groupby(['id'], as_index=False)['value'].max()  
    st_debt_repayments_agg.columns = ['id', 'st_debt_repayments']
    del st_debt_repayments_rows
    
    #get long term debt repayments  from cash flow stmt
    lt_debt_repayments_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        & (df.plabel.str.contains('long',na=False))
        & (df.plabel.str.contains('debt',na=False)) 
        & (df.plabel.str.contains('repayment',na=False))
    ]
    lt_debt_repayments_agg = lt_debt_repayments_rows.groupby(['id'], as_index=False)['value'].max()  
    lt_debt_repayments_agg.columns = ['id', 'lt_debt_repayments']
    del lt_debt_repayments_rows
    
    #try to get capital expenditure from cash flows 
    
    #get cash and cash equivalents
    cash_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('cash',na=False)) 
        ### exclusion words #####
        & (~df.plabel.str.contains('discontinue', na=False))
        & (~df.plabel.str.contains('cost', na=False))
        & (~df.plabel.str.contains('lease', na=False))
        & (~df.plabel.str.contains('adjustments', na=False))
        & (~df.plabel.str.contains('other', na=False))
        & (~df.plabel.str.contains('disposal', na=False))
        & (~df.plabel.str.contains('escrow', na=False))
        & (~df.plabel.str.contains('investing', na=False))
        & (~df.plabel.str.contains('partnership', na=False))
        & (~df.plabel.str.contains('restricted', na=False))
        & (~df.plabel.str.contains('securities', na=False))
    ]
    cash_agg = cash_rows.groupby(['id'], as_index=False)['value'].max()  
    cash_agg.columns = ['id', 'cash']
    del cash_rows
    
    #######################################################################################################
    #add financials to key_df
    merged_df = key_df.merge(cfo_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(revenue_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(shares_outstanding_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(current_assets_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(current_liabilities_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(dep_amort_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(dep_only_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(amort_only_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_prev_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(operating_income_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(lt_debt_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(st_debt_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(lt_debt_proceeds_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(st_debt_proceeds_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(lt_debt_repayments_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(st_debt_repayments_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(cash_agg, on = ['id'], how = "left")
    
    
    #######################################################################################################
    #delete unneeded things 
    del cfo_agg
    del revenue_agg
    del shares_outstanding_agg
    del current_assets_agg
    del current_liabilities_agg
    del dep_amort_agg
    del dep_only_agg
    del amort_only_agg
    del ppe_agg
    del ppe_prev_agg
    del operating_income_agg
    del lt_debt_agg
    del st_debt_agg
    del lt_debt_proceeds_agg
    del st_debt_proceeds_agg
    del lt_debt_repayments_agg
    del st_debt_repayments_agg
    del cash_agg 
    
    
    #print(merged_df)
    print(merged_df.columns)
    return(merged_df)
    #print(key_df)

#     return(result)
    #get all submissions 


In [7]:
# process the data in chunks first, return ticker level dataset 
def process_data():
    
    processed_list = []
    
    chunksize = 10 ** 5
    for chunk in pd.read_csv("total_finance_table.csv", chunksize=chunksize):  
        #for each chunk, get finance info on a company, yearly level
        processed_list.append(parse_data(chunk))
    #turn list of dataframes into one dataframe
    processed_table = pd.concat(processed_list)
    
    #split id back into year and cik
    processed_table[['year','cik']] = processed_table.id.str.split(pat = "_", expand=True) 
    del processed_list
    #######################################################################################################
    #aggregate up across chunks 
    agg_df = sqldf(
    """
    SELECT  
    year,
    CAST(cik AS INT64) as cik,
    MAX(revenue) AS revenue, 
    MAX(cfo) AS cfo, 
    MAX(shares_outstanding) AS shares_outstanding, 
    MAX(current_assets) AS current_assets,
    MAX(current_liabilities) AS current_liabilities,
    MAX(dep_amort) AS dep_amort,
    MAX(dep_only) AS dep_only,
    MAX(amort_only) AS amort_only,
    MAX(ppe) AS ppe,
    MAX(ppe_prev) AS ppe_prev,
    MAX(operating_income) AS operating_income,
    MAX(lt_debt) AS lt_debt,
    MAX(st_debt) AS st_debt,
    MAX(lt_debt_proceeds) AS lt_debt_proceeds,
    MAX(st_debt_proceeds) AS st_debt_proceeds,
    MAX(lt_debt_repayments) AS lt_debt_repayments,
    MAX(st_debt_repayments) AS st_debt_repayments,
    MAX(cash) AS cash 
    FROM 
    processed_table 
    GROUP BY     
    year,
    cik
    ORDER BY 2 DESC
    """)
    
    del processed_table 
    print(agg_df.columns)
    
    #######################################################################################################
    # turn long data wide 
    wide_df = agg_df.pivot(index='cik', columns='year', values=['revenue','cfo', 'shares_outstanding', 'current_assets',
                                                                'current_liabilities', 'dep_amort', 'dep_only', 'amort_only', 
                                                                'ppe','ppe_prev', 'operating_income', 
                                                                'lt_debt', 'st_debt',
                                                                'lt_debt_proceeds', 'st_debt_proceeds', 
                                                                'lt_debt_repayments', 'st_debt_repayments', 
                                                                'cash'])
    
    del agg_df
    
    #fix columns
    wide_df.columns = list(map("".join, wide_df.columns))
    
    # do some feature engineering and logic for data quality issues 
    #######################################################################################################
    
    
    
    wide_df.fillna(-1, inplace = True)
    
    #add in tickers
    wide_df = wide_df.merge(sample, on = ['cik'])
    
    #write to csv
    print(wide_df)
    wide_df.to_csv("sp500_financials.csv")
    return(wide_df)
    
start = time.time()

print("starting to process data")
p_df = process_data()

end = time.time()
print("finished processing data")
print(end - start)

starting to process data
Index(['id', 'cfo', 'revenue', 'shares_outstanding', 'current_assets',
       'current_liabilities', 'dep_amort', 'dep_only', 'amort_only', 'ppe',
       'ppe_prev', 'operating_income', 'lt_debt', 'st_debt',
       'lt_debt_proceeds', 'st_debt_proceeds', 'lt_debt_repayments',
       'st_debt_repayments', 'cash'],
      dtype='object')
Index(['id', 'cfo', 'revenue', 'shares_outstanding', 'current_assets',
       'current_liabilities', 'dep_amort', 'dep_only', 'amort_only', 'ppe',
       'ppe_prev', 'operating_income', 'lt_debt', 'st_debt',
       'lt_debt_proceeds', 'st_debt_proceeds', 'lt_debt_repayments',
       'st_debt_repayments', 'cash'],
      dtype='object')
Index(['id', 'cfo', 'revenue', 'shares_outstanding', 'current_assets',
       'current_liabilities', 'dep_amort', 'dep_only', 'amort_only', 'ppe',
       'ppe_prev', 'operating_income', 'lt_debt', 'st_debt',
       'lt_debt_proceeds', 'st_debt_proceeds', 'lt_debt_repayments',
       'st_debt_repaymen

finished processing data
243.36042141914368


In [8]:
print(p_df.columns)
print(p_df)

Index(['cik', 'revenue1218', 'revenue2013', 'revenue2014', 'revenue2015',
       'revenue2016', 'revenue2017', 'revenue2018', 'revenue2019',
       'revenue2020',
       ...
       'cash1218', 'cash2013', 'cash2014', 'cash2015', 'cash2016', 'cash2017',
       'cash2018', 'cash2019', 'cash2020', 'ticker'],
      dtype='object', length=164)
         cik  revenue1218   revenue2013   revenue2014   revenue2015  \
0       1800           -1  2.184800e+10  2.024700e+10  2.040500e+10   
1       2488           -1  5.299000e+09  5.506000e+09  3.991000e+09   
2       2969           -1 -1.000000e+00  1.043900e+10  9.894900e+09   
3       4127           -1 -1.000000e+00  2.291500e+09  3.258400e+09   
4       4281           -1  2.303200e+10  2.390600e+10  2.253400e+10   
..       ...          ...           ...           ...           ...   
493  1754301           -1 -1.000000e+00 -1.000000e+00 -1.000000e+00   
494  1754301           -1 -1.000000e+00 -1.000000e+00 -1.000000e+00   
495  1755672        

In [9]:
#get other ticker info from other script (current price, dividend yield, market cap, shares outsanding , sector)

In [10]:
#get 2018 walmart ticker = wmt, fy = 2018
# analyze missing info 
dq_df = pd.read_csv("total_finance_table.csv")

In [11]:
dq_df.columns

Index(['Unnamed: 0', 'name', 'sic', 'fye', 'form', 'period', 'fy', 'fp',
       'filed', 'ticker', 'cik', 'ddate', 'qtrs', 'uom', 'value', 'adsh',
       'stmt', 'tag', 'version', 'plabel'],
      dtype='object')

In [12]:
ex = dq_df[(dq_df['ticker'] == 'dis')
           #& (dq_df['fy'] == 2015.0)
           
           ]

In [13]:
for index, row in ex.iterrows():
    print(row)

Unnamed: 0                    1143
name                WALT DISNEY CO
sic                           7990
fye                            930
form                          10-K
period                 2.01909e+07
fy                            2019
fp                              FY
filed                     20191120
ticker                         dis
cik                        1744489
ddate                     20190930
qtrs                             0
uom                            USD
value                     9.79e+08
adsh          0001744489-19-000225
stmt                            BS
tag             OtherAssetsCurrent
version               us-gaap/2018
plabel        Other current assets
Name: 1533208, dtype: object
Unnamed: 0                    1144
name                WALT DISNEY CO
sic                           7990
fye                            930
form                          10-K
period                 2.01909e+07
fy                            2019
fp                        

Name: 1533325, dtype: object
Unnamed: 0                    1261
name                WALT DISNEY CO
sic                           7990
fye                            930
form                          10-K
period                 2.01909e+07
fy                            2019
fp                              FY
filed                     20191120
ticker                         dis
cik                        1744489
ddate                     20180930
qtrs                             4
uom                            USD
value                    2.503e+09
adsh          0001744489-19-000225
stmt                            CF
tag                IncomeTaxesPaid
version               us-gaap/2018
plabel           Income taxes paid
Name: 1533326, dtype: object
Unnamed: 0                    1262
name                WALT DISNEY CO
sic                           7990
fye                            930
form                          10-K
period                 2.01909e+07
fy                            20

Name: 1535733, dtype: object
Unnamed: 0                     3674
name                 WALT DISNEY CO
sic                            7990
fye                             930
form                           10-K
period                  2.01909e+07
fy                             2019
fp                               FY
filed                      20191120
ticker                          dis
cik                         1744489
ddate                      20180630
qtrs                              1
uom                             USD
value                          1.96
adsh           0001744489-19-000225
stmt                             IS
tag           EarningsPerShareBasic
version                us-gaap/2018
plabel                        Basic
Name: 1535734, dtype: object
Unnamed: 0                     3675
name                 WALT DISNEY CO
sic                            7990
fye                             930
form                           10-K
period                  2.01909e+07
fy    